# Federal Project Inventory Model Creation
After successful experimentation (see fpi_experiments.ipynb), for our each category, let's create and store (pickle) a model to identify them.

## Key Imports

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline

## Import our Labeled Text into a DataFrame

In [2]:
# Read the CSV File into a dataframe

labeled_data = pd.read_csv('Federal_Program_Inventory_Pilot_Data.csv',encoding='cp1252',
                           usecols=['Agency','Program Name','Program Description','Mission/Purpose',
                                    'Recipients','Beneficiaries','Associated Categories'])

# Remove duplicate rows
labeled_data = labeled_data.drop_duplicates().reset_index(drop=True)

# Get the categories
categories = pd.read_csv('Federal_Program_Inventory_Pilot_Data.csv',encoding='cp1252',usecols=['Category'])
categories = categories.drop_duplicates().reset_index(drop=True)
categories = [r["Category"] for _, r in categories.iterrows()]
categories

['Broadband',
 'Economic Development',
 'Opioid Epidemic Response',
 'STEM Education',
 'Workforce Development',
 'Native American',
 'Flood Risk',
 'A.I. R&D/Quantum R&D',
 'Global Health',
 'Homelessness',
 'HIV/AIDS',
 'Transportation Infrastructure']

## Define a function to use GridSearchCV to find a good estimator

In [3]:
def find_estimator(data):
    """
    Given a dataframe with a column 'text' and a binary 'label', use GridSearchCV
    to find an estimator that fits
    """
    # Note: Removed loss functions that didn't support predict_proba
    param_dist = {
      "vectorizer__ngram_range":[(1,1), (1,2)],
      "select__k":[10,50,100,200,500,1000,1500],
#      "classifier__loss": ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron']
      "classifier__loss": ['log', 'modified_huber']
    }

    pipe = Pipeline([('vectorizer', TfidfVectorizer(stop_words='english')),
              ('select', SelectKBest()),
              ('classifier', SGDClassifier())])

    search = GridSearchCV(pipe, 
                          param_grid = param_dist,
                          cv = 5,
                          n_jobs = -1,
                          verbose = 1)


    search.fit(data['text'],data['label']);
    return search.best_estimator_

def find_estimator_iterate(data, times):
    """
    Splits the data multiple times to find the best estimator
    """
    best_f1 = 0
    for i in range(times):
        # Do an 80/20 split to find training data
        training_data, test_data = train_test_split(data, test_size=0.2, stratify=data['label'])
        # Find the estimator
        estimator = find_estimator(training_data)
        # Calculate the f1 score on test data
        pred = estimator.predict(test_data['text'])
        f1_score = metrics.f1_score(test_data['label'],pred,average='macro')
        print(f1_score)
        # Replace if better
        if f1_score > best_f1:
            best_f1 = f1_score
            best_estimator = estimator
            best_test_data = test_data
    # Print the best f1 score
    pred = best_estimator.predict(best_test_data['text'])
    print(metrics.classification_report(best_test_data['label'],pred))
    return best_estimator

        

## Now lets use the function to find estimators for all 12 categories

In [4]:
# For our first test, let's join the Agency, Program Name, Program Description, Mission/Purpose, and Recipients 
# into a single Text field and see if we can determine "Broadband"
labeled_data['text'] = labeled_data[['Agency','Program Name','Program Description','Mission/Purpose',
                                    'Recipients','Beneficiaries']].agg(' '.join, axis=1)

labeled_data['categories'] = labeled_data['Associated Categories'].str.split('; ')

# Place to save our estimators
estimators = {}

# Iterate over labels
for pos_label in categories:
    display("Category = " + pos_label)
    
    # Let's label the items in the category (here we're cheating and doing a simple substring)
    labeled_data['label'] = np.where(labeled_data['Associated Categories'].str.contains(pos_label),'Y','N')

    # Display the first 6 rows to show that we successfully read it
    # display(labeled_data.head(6))

    # Now get the estimator
    estimators[pos_label] = find_estimator_iterate(labeled_data,10)


'Category = Broadband'

Fitting 5 folds for each of 28 candidates, totalling 140 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 122 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 140 out of 140 | elapsed:    5.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


0.8944649446494465
Fitting 5 folds for each of 28 candidates, totalling 140 fits


[Parallel(n_jobs=-1)]: Done 140 out of 140 | elapsed:    4.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


...

[Parallel(n_jobs=-1)]: Done 140 out of 140 | elapsed:    4.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


0.9061884977039143
Fitting 5 folds for each of 28 candidates, totalling 140 fits


[Parallel(n_jobs=-1)]: Done 140 out of 140 | elapsed:    4.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


0.9687294992346381
Fitting 5 folds for each of 28 candidates, totalling 140 fits
0.9648216482164822
              precision    recall  f1-score   support

           N       1.00      1.00      1.00       135
           Y       1.00      1.00      1.00         8

    accuracy                           1.00       143
   macro avg       1.00      1.00      1.00       143
weighted avg       1.00      1.00      1.00       143



[Parallel(n_jobs=-1)]: Done 140 out of 140 | elapsed:    4.4s finished


In [5]:
estimators

{'Broadband': Pipeline(steps=[('vectorizer',
                  TfidfVectorizer(ngram_range=(1, 2), stop_words='english')),
                 ('select', SelectKBest()),
                 ('classifier', SGDClassifier(loss='modified_huber'))]),
 'Economic Development': Pipeline(steps=[('vectorizer',
                  TfidfVectorizer(ngram_range=(1, 2), stop_words='english')),
                 ('select', SelectKBest(k=500)),
                 ('classifier', SGDClassifier(loss='modified_huber'))]),
 'Opioid Epidemic Response': Pipeline(steps=[('vectorizer',
                  TfidfVectorizer(ngram_range=(1, 2), stop_words='english')),
                 ('select', SelectKBest()),
                 ('classifier', SGDClassifier(loss='modified_huber'))]),
 'STEM Education': Pipeline(steps=[('vectorizer',
                  TfidfVectorizer(ngram_range=(1, 2), stop_words='english')),
                 ('select', SelectKBest(k=500)),
                 ('classifier', SGDClassifier(loss='modified_huber'))]),

## Test the estimators on an example

In [6]:
def predict_categories(text,estimators):
    """
    Given text and a dictionary of estimators, apply the estimators on the text and sort the result
    by the calculated probability
    """
    results = []
    for category, estimator in estimators.items():
        results.append((category, estimator.predict_proba([text])[0][1]))
    return sorted(results, key = lambda x: -x[1])
        

In [7]:
predict_categories("This is a test of homelessness and broadband", estimators)

[('Broadband', 1.0),
 ('Homelessness', 1.0),
 ('Opioid Epidemic Response', 0.06096071424030308),
 ('Workforce Development', 0.04613192362603553),
 ('HIV/AIDS', 0.03799687034513687),
 ('Native American', 0.03140310219771464),
 ('Economic Development', 0.0),
 ('STEM Education', 0.0),
 ('Flood Risk', 0.0),
 ('A.I. R&D/Quantum R&D', 0.0),
 ('Global Health', 0.0),
 ('Transportation Infrastructure', 0.0)]

## Pickle the estimators

In [8]:
import pickle

pickle.dump(estimators, open("fpi_estimators.pkl", "wb"))

## Now load and test the estimators

In [9]:
import pickle

estimators = pickle.load(open("fpi_estimators.pkl", "rb"))

In [10]:
predict_categories("This is a test of economic development and native americans", estimators)

[('Native American', 0.6586695257553425),
 ('Economic Development', 0.2937178743348636),
 ('Opioid Epidemic Response', 0.06096071424030308),
 ('Workforce Development', 0.04613192362603553),
 ('HIV/AIDS', 0.024041584283893518),
 ('Homelessness', 0.023516401253228236),
 ('Broadband', 0.01319442303341195),
 ('STEM Education', 0.0),
 ('Flood Risk', 0.0),
 ('A.I. R&D/Quantum R&D', 0.0),
 ('Global Health', 0.0),
 ('Transportation Infrastructure', 0.0)]